# Fire Predictions From CNN

Loads a CNN model from S3, applies raw input data to the model, and makes predictions.

In [221]:
# Load packages
import boto3
import datetime
import io
import itertools
import keras
import numpy as np
import os
import pickle
import rasterio as rio
import requests

In [222]:
# s3 config
s3_client = boto3.client('s3')
bucket_name = 'hotzone'

# matrix config
matrix_dim = 32

## Load Model from S3

In [223]:
def pull_data_from_s3(s3_client, bucket_name, key_name):
    '''
    Pulls pre-processed data from S3.

    Args:
        - s3_client: boto3 s3 client
        - bucket_name: name of bucket on s3 to pull data from
        - key_name: directory/file_name to pull data from
    Returns:
        - Nothing
    
    https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
    '''
    
    array_data = io.BytesIO()
    s3_client.download_fileobj(bucket_name, key_name, array_data)
    
    array_data.seek(0)
    array = pickle.load(array_data)

    return array

In [224]:
# load model from s3
new_config = pull_data_from_s3(s3_client, bucket_name, 'models/model_config.pickle')
new_weights = pull_data_from_s3(s3_client, bucket_name, 'models/model_weights.pickle')

model = keras.Model.from_config(new_config)
model.set_weights(new_weights)

## Make Predictions

In [225]:
lat_long_coords = [
    (-120.63089414255178, 38.86743631001172),
    (-120.62207064002624, 38.86743631001172),
    (-120.63089414255178, 38.86326036507092),
    (-120.61765888876347, 38.86326036507092),
    (-120.63530589381455, 38.859084420130124),
    (-120.60442363497516, 38.859084420130124),
    (-120.63971764507731, 38.854908475189326),
    (-120.6000118837124, 38.854908475189326),
    (-120.64412939634009, 38.85073253024853),
    (-120.6000118837124, 38.85073253024853),
    (-120.63530589381455, 38.84655658530773),
    (-120.58677662992409, 38.84655658530773),
    (-120.62207064002624, 38.84238064036694),
    (-120.56912962487301, 38.84238064036694),
    (-120.61765888876347, 38.83820469542614),
    (-120.61324713750071, 38.83820469542614),
    (-120.59560013244963, 38.83820469542614),
    (-120.56471787361025, 38.83820469542614),
    (-120.59560013244963, 38.834028750485345),
    (-120.56030612234748, 38.834028750485345),
    (-120.5558943710847, 38.834028750485345),
    (-120.55148261982194, 38.834028750485345),
    (-120.64412939634009, 38.82985280554455),
    (-120.63971764507731, 38.82985280554455),
    (-120.59118838118687, 38.82985280554455),
    (-120.54707086855917, 38.82985280554455),
    (-120.64412939634009, 38.82567686060375),
    (-120.63089414255178, 38.82567686060375),
    (-120.6000118837124, 38.82567686060375),
    (-120.53824736603363, 38.82567686060375),
    (-120.64854114760286, 38.82150091566295),
    (-120.54707086855917, 38.82150091566295),
    (-120.53824736603363, 38.82150091566295),
    (-120.53383561477087, 38.82150091566295),
    (-120.68824690896777, 38.817324970722154),
    (-120.683835157705, 38.817324970722154),
    (-120.6573646501284, 38.817324970722154),
    (-120.53824736603363, 38.817324970722154),
    (-120.683835157705, 38.81314902578136),
    (-120.6705999039167, 38.81314902578136),
    (-120.66618815265393, 38.81314902578136),
    (-120.5294238635081, 38.81314902578136),
    (-120.67942340644224, 38.80897308084056),
    (-120.67501165517947, 38.80897308084056),
    (-120.66177640139117, 38.80897308084056),
    (-120.5294238635081, 38.80897308084056),
    (-120.6705999039167, 38.80479713589976),
    (-120.5294238635081, 38.80479713589976),
    (-120.683835157705, 38.800621190958964),
    (-120.58236487866132, 38.800621190958964),
    (-120.57795312739856, 38.800621190958964),
    (-120.51618860971979, 38.800621190958964),
    (-120.67501165517947, 38.796445246018166),
    (-120.66177640139117, 38.796445246018166),
    (-120.64854114760286, 38.796445246018166),
    (-120.59560013244963, 38.796445246018166),
    (-120.58236487866132, 38.796445246018166),
    (-120.53824736603363, 38.796445246018166),
    (-120.5294238635081, 38.796445246018166),
    (-120.51177685845703, 38.796445246018166),
    (-120.683835157705, 38.79226930107737),
    (-120.66618815265393, 38.79226930107737),
    (-120.61324713750071, 38.79226930107737),
    (-120.6000118837124, 38.79226930107737),
    (-120.57795312739856, 38.79226930107737),
    (-120.50295335593148, 38.79226930107737),
    (-120.60442363497516, 38.78809335613657),
    (-120.59560013244963, 38.78809335613657),
    (-120.58677662992409, 38.78809335613657),
    (-120.48971810214319, 38.78809335613657),
    (-120.60442363497516, 38.78391741119577),
    (-120.49412985340595, 38.78391741119577),
    (-120.58677662992409, 38.779741466254976),
    (-120.48971810214319, 38.779741466254976),
    (-120.56471787361025, 38.775565521314185),
    (-120.50736510719426, 38.775565521314185),
    (-120.54707086855917, 38.77138957637339),
    (-120.50736510719426, 38.77138957637339)
]

max_values = {
    'rainint': 81.55091806687825, 
    'High T': 317.42411008927, 
    'Low T': 300.3468885291001, 
    'Humidity': 1.0, 
    'Wind Speed': 46.35018731657426, 
    'Wind Direction': 358.842808170917, 
    'Fire Direction': 8, 
    'Fire Speed': 6
}

In [226]:
def get_index(long,lat):
    
    """Get the coordinates of a given pixel in the tif coordinate
    system
    Input: geotiff path+filename, and row,column of the pixel in question
    Output: tuple of the x,y coordinate in the geotif coordinate system
    """

    left = -123.50294421461426
    top = 39.00106654811723

    xres = 0.004411751262768785
    yres = -0.0041759449407971815
    
    pixelcol = int(np.rint((long - left)/xres))
    pixelrow = int(np.rint((lat - top)/yres))
    
    return (pixelrow, pixelcol)

In [227]:
def get_coords(y, x):
    
    """Get the coordinates of a given pixel in the tif coordinate
    system
    Input: geotiff path+filename, and row,column of the pixel in question
    Output: tuple of the x,y coordinate in the geotif coordinate system
    """
    
    left = -123.50294421461426
    top = 39.00106654811723

    xres = 0.004411751262768785
    yres = -0.0041759449407971815

    deltax = xres*x
    deltay = yres*y

    x = left+deltax
    y = top+deltay
    
    return (x,y)

In [228]:
def get_tomorrow_weather(max_values, lat, long):
    
    s = requests.Session()
    s.auth = ('user', 'pass')
    s.headers.update({'Accept-Encoding':'gzip'})
    headers = {'Accept-Encoding':'gzip'}
    
    key = '5ffac5f056d341c6296cba58fa96e9ba'
    date = str(datetime.date.today() + datetime.timedelta(days = 1)) + 'T12:00:00'
    lat = str(lat,)
    long = str(long)
    blocks = '[currently,minutely,hourly,alerts]'
    units = 'ca'

    # set the query string for darksky
    query = ('https://api.darksky.net/forecast/'+key+'/'+ 
    lat+','+long+','+date+'?exclude=' 
    +blocks+'&units='+units)

    # Make the call to Dark Sky to get all the data for that date and location
    r = s.get(query,headers=headers)

    data = r.json()
    data = data['daily']['data'][0]
    
    rainint = data['precipIntensityMax']
    high_t = data['temperatureHigh']
    low_t= data['temperatureLow']
    humidity = data['humidity']
    wind_speed = data['windSpeed']
    wind_direction = data['windBearing']
    
    weather_data = {
        'rainint': rainint,
        'High T': high_t,
        'Low T': low_t,
        'Humidity': humidity,
        'Wind Speed': wind_speed,
        'Wind Direction': wind_direction,
        'Fire Direction': 8,
        'Fire Speed': 6
    }
    
    weather = []
        
    for k, v in weather_data.items():
        max_val = max_values.get(k, 1)
        
        val = v/float(max_val)
        weather.append(val)
          
    return weather

In [229]:
def predict_next_day(lat_long_coords):

    today = np.zeros((719, 908))
    side = matrix_dim/2
    
    values = []
    
    for (long, lat) in lat_long_coords:
        index = get_index(long,lat)
        today[index] = 1
        
    np.pad(today, pad_width=matrix_dim, mode='constant', constant_values=0)
    
    vals = np.where(today == 1)
    
    x_avg = int(np.mean(vals[0]))
    y_avg = int(np.mean(vals[1]))

    x_min = x_avg - 50
    x_max = x_avg + 50
    
    x_min = max(x_min, 0)
    x_max = min(x_max, 972)
    
    y_min = y_avg - 50
    y_max = y_avg + 50
    
    y_min = max(y_min, 0)
    y_max = min(y_max, 783)
    
    
    x_vals = range(x_min, x_max)
    y_vals = range(y_min, y_max)
    
    vals = list(itertools.product(x_vals, y_vals))
    
    values = []
    
    shape = today.shape
    prediction = np.zeros(shape)
    
    (long, lat) = get_coords(x_avg, y_avg)
    
    weather = get_tomorrow_weather(max_values, lat, long)
    
    for (xi, yi) in vals:
        
        point = (xi, yi)

        xi_r = int(xi + side)
        xi_l = int(xi - side)
        yi_b = int(yi + side)
        yi_t = int(yi - side)
        
        if xi_r > 0 and xi_l > 0 and yi_b > 0 and yi_t > 0:

            m = today[xi_l:xi_r, yi_t:yi_b]

            if (m.shape == (matrix_dim, matrix_dim)):
                values.append((point, weather, m))
                

    for (point, w, f) in values:        
        fire = []
        weather = []

        fire.append(np.asarray(f))
        weather.append(np.asarray(w))

        fire = np.asarray(fire)
        weather = np.asarray(weather)
        
        obs = len(fire)
        fire = fire.reshape(obs, matrix_dim, matrix_dim, 1)

        val = model.predict([fire, weather])
            
        prediction[point] = val
    
    return prediction

In [235]:
def calculate_coords(matrix):
    
    outline = np.rint(matrix)
    outline = np.diff(outline)
    outline = np.abs(outline)

    # get pixels from outline
    poly_to_plot = np.where(outline != 0)

    # instantiate a matrix in the target shape
    shape = outline.shape
    poly = np.zeros(shape)

    # create a list of coordinates in the tif coordinate system
    tif_coordinates = []

    for (xi, yi) in list(zip(poly_to_plot[0], poly_to_plot[1])):
        poly[xi,yi] = 1
        coords = get_coords(xi, yi)
        tif_coordinates.append(coords)

    return tif_coordinates

In [236]:
values = predict_next_day(lat_long_coords)
coords = calculate_coords(values)